In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Measurement Conditions

In [2]:
d = 10 #depth
r_SAD = 10 #field size
SAD = 100 #SAD
PDD = 66.4 # if required, obtain from interpolation data
'''Lecture 8, Slide 13, for computing kQ'''
A = 0.9652
B = 2.141e-3
C = -2.623e-5

# Theory

For a dose (D) in water (W) to a phantom we have

$$D_W^Q = Mk_Q N_{D,W}^{^{60}Co}$$

* $M$ is the **corrected electrometer reading**
* $k_Q$ is the beam quality factor (relates arbitrary beam to cobalt, for cobalt $=1$)
* $N_{D,W}^{^{60}Co}$ is a calibration constant for a Co-60 beam under reference conditions

In [3]:
N60Co = 1.1067

Obtain kQ

In [4]:
kQ = A+B*PDD+C*PDD**2
print(f'kQ={kQ:.4f}')

kQ=0.9917


To obtain the **corrected electrometer reading** $M$, find this using

$$ M = P_{ion}P_{TP}P_{pol}M_{raw} $$

* Ion recombination factor estimated using low and high voltages $$P_{ion} = \left(1-\frac{V_H}{V_L}\right)/\left(\frac{M_{raw}^H}{M_{raw}^L - V_H/V_L}\right)$$
* Polarization correction factor estimated using positive and negative voltages
$$P_{pol} = \frac{M_{raw}^+-M_{raw}^-}{2M_{raw}^+}$$
* Temperature pressure correction factor estimated as follows:
$$P_{TP} = \frac{T_w}{295.2}\frac{760}{P_{air}}$$

**Note**: $H$ corresponds to voltage readings at +300V, $L$ corresponds to voltage readings at 150V, and $\pm$ corresponds to $\pm$300V

## Compute P_TP

In [5]:
T_w = (20.6+273.15) #K
P_air = 743.7 #mmHg

Compute $P_{TP}$

In [6]:
PTP = (T_w/295.2)*(760/P_air)
print(f'PTP={PTP:.4f}')

PTP=1.0169


## Compute P_pol and P_ion

Get voltage readings:

In [7]:
Voltages = {
    '+300V': {'Voltage':320, 'Readings':np.array([138.6, 138.7])},
    '+150V': {'Voltage':160.3, 'Readings':np.array([138.2, 138.3])},
    '-300V': {'Voltage':-319.6, 'Readings':np.array([-138.6, -138.6])},
}

Use these to compute $P_{pol}$ and $P_{ion}$

In [8]:
Mraw_p = Mraw_h = np.mean(Voltages['+300V']['Readings'])
Mraw_l = np.mean(Voltages['+150V']['Readings'])
Mraw_n = np.mean(Voltages['-300V']['Readings'])
VH = Voltages['+300V']['Voltage']
VL = Voltages['+150V']['Voltage']
Ppol = (Mraw_p - Mraw_n)/(2*Mraw_p)
Pion = (1-VH/VL)/(Mraw_h/Mraw_l - VH/VL)
print(f'Ppol={Ppol:.4f}')
print(f'Pion={Pion:.4f}')

Ppol=0.9998
Pion=1.0029


## Compute M

In [9]:
Mraw = Mraw_p
M = Pion*Ppol*PTP*Mraw
print(f'M={M:.2f}')

M=141.38


## Compute Dose

In [10]:
D = M*kQ*N60Co
print(f'D={D:.2f}Gy')

D=155.17Gy
